In [6]:
# 具有分類路徑、出版日期與ISBN13的爬蟲程式
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import re
import traceback
from datetime import datetime

# 設置日誌記錄（使用日期格式）
log_date = datetime.now().strftime('%Y%m%d')
log_filename = f'C:/Users/HP/Desktop/project/book-ranking/實體書排行榜code/log/sanmin_plog_{log_date}.txt'
logging.basicConfig(filename=log_filename, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# 基本設置
base_url = "https://www.sanmin.com.tw/promote/top/?id=yy&item=11410"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 使用 session 來減少重複連線開銷
session = requests.Session()
data = []

# 設置自動重試機制
def fetch_with_retry(url, max_retries=3):
    """嘗試請求 URL，最多重試 max_retries 次"""
    for attempt in range(max_retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return response
            else:
                print(f"⚠️ Attempt {attempt+1} failed: {response.status_code}")
                time.sleep(2)  # 延遲再試
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Attempt {attempt+1} failed: {e}")
            time.sleep(2)
    return None  # 如果三次都失敗，返回 None

# 爬取每一頁的資料
for page in range(1, 14):
    url = f"{base_url}&pi={page}"
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    logging.info(f"Processing page {page}")
    print(f"Processing page {page}")

    # 查找所有產品資訊
    products = soup.find_all('div', class_='sProduct')

    for product in products:
        try:
            rank = product.find('div', class_='Title').get_text(strip=True).split('.')[0]
            name = product.find('h3').get_text(strip=True)

            # 提取商品ID
            product_classes = product.get('class', [])
            product_id = None
            
            # 方法1：從 class 屬性中找到以 'Prod' 開頭且後面跟數字的類名
            for cls in product_classes:
                # 必須是 'Prod' + 數字的格式，例如 'Prod1234567'
                if cls.startswith('Prod') and len(cls) > 4 and cls[4:].isdigit():
                    product_id = cls[4:]  # 取 'Prod' 之後的數字部分
                    break
            
            # 方法2：如果找不到，嘗試從連結中提取
            if not product_id:
                link_tag = product.find('a', href=True)
                if link_tag and '/product/' in link_tag['href']:
                    # 從 URL 中提取 ID
                    match = re.search(r'/product/index/(\d+)', link_tag['href'])
                    if match:
                        product_id = match.group(1)
            
            # 方法3：嘗試從所有連結中提取
            if not product_id:
                all_links = product.find_all('a', href=True)
                for link in all_links:
                    match = re.search(r'/product/index/(\d+)', link['href'])
                    if match:
                        product_id = match.group(1)
                        break
            
            if not product_id:
                logging.warning(f"無法提取商品ID，跳過此商品: {name}")
                print(f"⚠️ 無法提取商品ID，跳過: {name}")
                continue
            
            detail_url = f"https://www.sanmin.com.tw/product/index/{product_id}"

            # 提取價格並清理格式（從列表頁）
            price_tag = product.select_one('.Price')
            if price_tag:
                price_text = price_tag.get_text(strip=True)
                # 移除非數字字符（保留小數點）
                price = re.sub(r'[^\d.]', '', price_text)
                if not price:
                    price = '未知'
            else:
                price = '未知'

            # 訪問商品詳細頁面
            detail_response = fetch_with_retry(detail_url)
            if detail_response is None:
                print(f"❌ Failed to fetch {detail_url} after retries")
                logging.error(f"Failed to fetch {detail_url} after retries")
                continue  # 跳過此書，避免影響後續爬取

            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')


            # 提取分類路徑
            breadcrumb_tags = detail_soup.select('#breadcrumb-trail a')  # 直接選擇所有 <a> 標籤
            category_path = '-'.join([tag.get_text(strip=True) for tag in breadcrumb_tags if tag.get_text(strip=True) != "三民網路書店"]) if breadcrumb_tags else '未知'
            
            # 提取副分類（中文圖書分類）
            try:
                vice_category = ''
                # 查找所有 div.item
                item_divs = detail_soup.find_all('div', class_='item')
                for item_div in item_divs:
                    # 查找是否包含"中文圖書分類"
                    h2_tag = item_div.find('h2', class_='d-inline fs-14')
                    if h2_tag and '中文圖書分類' in h2_tag.get_text():
                        # 提取 h3 中的連結文本
                        h3_tag = item_div.find('h3', class_='d-inline fs-14')
                        if h3_tag:
                            link_tag = h3_tag.find('a', class_='text-blue bold')
                            if link_tag:
                                vice_category = link_tag.get_text(strip=True)
                                break
            except Exception as e:
                logging.error(f"Error extracting vice_category for product {product_id}: {e}")
                vice_category = ''
            
            # 一次性查找所有 li.ga 標籤，供多個字段使用
            li_tags = detail_soup.find_all('li', class_='ga')
            
            # 提取作者（從詳細頁）
            try:
                authors = ''
                for li in li_tags:
                    h3_tag = li.find('h3')
                    if h3_tag:
                        h3_text = h3_tag.get_text(strip=True)
                        # 檢查是否包含"作者"、"編者"、"著者"等關鍵字
                        if any(keyword in h3_text for keyword in ['作者', '編者', '著者', '編著']):
                            # 提取所有 a 標籤（作者可能有多個）
                            author_links = h3_tag.find_all('a')
                            if author_links:
                                authors = '; '.join([link.get_text(strip=True) for link in author_links])
                            else:
                                # 如果沒有 a 標籤，直接提取文本並清理
                                authors = re.sub(r'(作者|編者|著者|編著)[:：\s]*', '', h3_text).strip()
                            break
            except Exception as e:
                logging.error(f"Error extracting author for product {product_id}: {e}")
                authors = ''
            
            # 提取出版社（從詳細頁）
            try:
                publisher = ''
                for li in li_tags:
                    h3_tag = li.find('h3')
                    if h3_tag:
                        h3_text = h3_tag.get_text(strip=True)
                        # 檢查是否包含"出版社"關鍵字
                        if '出版社' in h3_text:
                            # 提取 a 標籤
                            pub_link = h3_tag.find('a')
                            if pub_link:
                                publisher = pub_link.get_text(strip=True)
                            else:
                                # 如果沒有 a 標籤，直接提取文本並清理
                                publisher = re.sub(r'出版社[:：\s]*', '', h3_text).strip()
                            break
            except Exception as e:
                logging.error(f"Error extracting publisher for product {product_id}: {e}")
                publisher = ''
            
            # 提取譯者（從詳細頁）
            try:
                translator = ''
                for li in li_tags:
                    h3_tag = li.find('h3')
                    if h3_tag:
                        h3_text = h3_tag.get_text(strip=True)
                        # 檢查是否包含"譯者"關鍵字
                        if '譯者' in h3_text:
                            # 提取所有 a 標籤（譯者可能有多個）
                            translator_links = h3_tag.find_all('a')
                            if translator_links:
                                translator = '; '.join([link.get_text(strip=True) for link in translator_links])
                            else:
                                # 如果沒有 a 標籤，直接提取文本並清理
                                translator = re.sub(r'譯者[:：\s]*', '', h3_text).strip()
                            break
            except Exception as e:
                logging.error(f"Error extracting translator for product {product_id}: {e}")
                translator = ''

            # 提取 ISBN13 並清理格式
            try:
                isbn13 = ''
                for li in li_tags:
                    if "ISBN13" in li.text:
                        # 提取 span 標籤中的 ISBN
                        isbn_span = li.find_all('span')
                        if len(isbn_span) >= 3:
                            isbn_text = isbn_span[-1].get_text(strip=True)
                        else:
                            isbn_text = li.get_text(strip=True).replace('ISBN13', '').replace('：', '').strip()
                        
                        # 移除破折號和空格
                        isbn13 = re.sub(r'[-\s]', '', isbn_text)
                        break
            except Exception as e:
                logging.error(f"Error extracting ISBN13 for product {product_id}: {e}")
                isbn13 = ''

            # 提取出版日期並轉換格式
            try:
                pub_date = ''
                for li in li_tags:
                    if "出版日" in li.text:
                        # 提取 span 標籤中的日期
                        date_span = li.find_all('span')
                        if len(date_span) >= 3:
                            pub_date_text = date_span[-1].get_text(strip=True)
                        else:
                            pub_date_text = li.get_text(strip=True).replace('出版日', '').replace('：', '').strip()
                        
                        # 轉換日期格式：YYYY/MM/DD 或 YYYY-MM-DD -> YYYY/M/D（去除前導0）
                        date_match = re.match(r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})', pub_date_text)
                        if date_match:
                            year = date_match.group(1)
                            month = str(int(date_match.group(2)))  # 去除前導0
                            day = str(int(date_match.group(3)))    # 去除前導0
                            pub_date = f"{year}/{month}/{day}"
                        else:
                            pub_date = pub_date_text
                        break
            except Exception as e:
                logging.error(f"Error extracting PubDate for product {product_id}: {e}")
                pub_date = ''
            
            # 提取原文書名（替代書名）
            try:
                original_title = ''
                for li in li_tags:
                    if "替代書名" in li.text:
                        # 提取 a 標籤或 span 標籤中的書名
                        title_link = li.find('a')
                        if title_link:
                            original_title = title_link.get_text(strip=True)
                        else:
                            title_span = li.find_all('span')
                            if len(title_span) >= 3:
                                original_title = title_span[-1].get_text(strip=True)
                        break
            except Exception as e:
                logging.error(f"Error extracting OriginalTitle for product {product_id}: {e}")
                original_title = ''

            # 提取多個定位關鍵字的內容
            schemes = []
            keywords = ["三民出版品", "親子館", "中文圖書分類", "得獎作品"]
            for scheme_type in keywords:
                scheme_tag = detail_soup.find('a', class_='text-secondary bold', text=scheme_type)
                if scheme_tag:
                    main_title = scheme_tag.get_text(strip=True)
                    sub_titles = scheme_tag.find_parent('div', class_='text-secondary py3').find_all('h3', class_='d-inline fs-14')
                    for sub_title in sub_titles:
                        sub_text = sub_title.get_text(strip=True)
                        schemes.append(f"{main_title}-{sub_text}")

            scheme_text = '; '.join(schemes) if schemes else '未知'
            
            # 處理標題：移除標點符號和空格
            processed_title = re.sub(r'[^\w]', '', name)

            # 添加資料
            data.append({
                "ISBN": isbn13,
                "production_id": product_id,
                "title": name,
                "processed_title": processed_title,
                "Publisher": publisher,
                "author": authors,
                "translator": translator,  # 從詳細頁提取
                "original_title": original_title,  # 從"替代書名"提取
                "publish_date": pub_date,
                "fixed_price": price,
                "category": category_path,
                "vice_category": vice_category,  # 中文圖書分類的子分類
                "url": detail_url,
                "Day_counts": "",
                "rank": rank  # 用於後面填充日期欄位
            })

            logging.info(f"✅ 排名 {rank}: {name} (ID: {product_id})")
            print(f"✅ 排名 {rank}: {name}")
        except AttributeError as e:
            logging.error(f"❌ 解析商品時發生錯誤: {e}")
            print(f"❌ 解析商品時發生錯誤: {e}")
            continue
        except Exception as e:
            logging.error(f"❌ 處理商品時發生未預期錯誤: {e}")
            print(f"❌ 處理商品時發生未預期錯誤: {e}")
            traceback.print_exc()
            continue

    # 暫停以避免被伺服器禁止
    time.sleep(2)

# 保存資料到CSV文件（金石堂格式）
df = pd.DataFrame(data)

# 獲取今天的日期（格式：11/25）
today = datetime.now()
date_column = f"{today.month}/{today.day}"

# 添加日期欄位並填充排名
df[date_column] = df['rank']
df = df.drop('rank', axis=1)

# 調整欄位順序（三民增加 vice_category）
column_order = ['ISBN', 'production_id', 'title', 'processed_title', 'Publisher', 
                'author', 'translator', 'original_title', 'publish_date', 'fixed_price', 
                'category', 'vice_category', 'url', 'Day_counts', date_column]
df = df[column_order]

# 使用日期命名檔案
file_date = today.strftime('%Y%m%d')
output_path = f'C:/Users/HP/Desktop/project/book-ranking/ranking_result/sanmin/sanmin_all_categories_{file_date}.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

logging.info(f"資料已保存到 {output_path}")
print(f"資料已保存到 {output_path}")
print(f"共爬取 {len(df)} 筆資料")



Processing page 1
✅ 排名 1: 親愛的末期癌症：蔡康永真情推薦！低谷總裁的二十堂生命體悟課


C:\Users\HP\AppData\Local\Temp\ipykernel_59584\1281126030.py:274: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  scheme_tag = detail_soup.find('a', class_='text-secondary bold', text=scheme_type)


✅ 排名 2: Amy Gets Eaten
✅ 排名 3: We Went to Find a Woolly Mammoth
✅ 排名 4: 主題百匯高中英文克漏字
✅ 排名 5: 古今悅讀一百
✅ 排名 6: 主題百匯高中英文閱讀測驗：篇章結構．閱讀測驗．混合題
✅ 排名 7: 原子習慣：細微改變帶來巨大成就的實證法則
✅ 排名 8: 中國仙道之究竟第一集：內金丹法
✅ 排名 9: 培育絕美鹿角蕨：進階栽培×配置設計，兼顧收藏與裝飾的綠植美學
✅ 排名 10: 2026年故宮國寶聚焦：書畫精品選粹大月曆
✅ 排名 11: 技術型高中英文第三冊職場多益TOEIC Up!(A版)(附解析夾冊)
✅ 排名 12: 學測歷屆試題：自然考科（107～114年）
✅ 排名 13: 國防產業地緣政治學【全民必備．第一本全球軍工產業分析全解讀】：從重工業、軟體業、航太業到AI技術轉型，揭祕軍工產業最新數據，看懂全球防衛布局，打造政經投資視野！
✅ 排名 14: 台灣AI大未來：解析最新的AI趨勢、台灣情勢、企業布局與個人發展
✅ 排名 15: 中共間諜戰術全解析：800 起真實案例，前美國中情局官員揭露全球共諜行動的面貌
✅ 排名 16: 對話式高中數學1-2冊學測複習講義
✅ 排名 17: 月考王：地理1
✅ 排名 18: 學測歷屆試題：國文（107～114年）
✅ 排名 19: 新譯老子讀本(四版)
✅ 排名 20: 人體透視書
✅ 排名 21: 小雲的飄浮日記
✅ 排名 22: 修仙寶典（天機秘文）
✅ 排名 23: 中國仙道之究竟第三集：女金丹法
✅ 排名 24: 技術型高中英文第一冊學習講義(A版)(附解析夾冊)
✅ 排名 25: 國家為什麼會破產：橋水基金應對大週期的原則
✅ 排名 26: 學測歷屆試題：英文（107～114年）
✅ 排名 27: 新譯古文觀止(上/下)(增訂五版)
✅ 排名 28: 神奇咒語咕哩咚
✅ 排名 29: 新基本小六法（2025年9月）
✅ 排名 30: 社會心理學
✅ 排名 31: 憲法釋論(修訂五版)
✅ 排名 32: 新譯孫子讀本(三版)
✅ 排名 33: 最新綜合六法全書(2025年9月版)
✅ 排名 34: 小學生國語辭典(增訂五版一刷)
✅ 排名 35: 中國仙道之究竟第二集：性命篇
✅ 排名 36: 領航高中地理1
✅ 排名